### SQL Langgraph Agent: 
- Get notification , contribution and information form the EU transparency register

In [2]:
import os
from sqlalchemy import create_engine
import pandas as pd

from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver

from langgraph.prebuilt import create_react_agent

In [3]:
from dotenv import find_dotenv, load_dotenv

_ = load_dotenv(find_dotenv())
OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
LANGCHAIN_API_KEY = os.getenv("LANGSMITH_API_KEY")

In [4]:
# initiate the langsmith for tracing the sql langgraph agent
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="tris project"

In [5]:
# this function is for tracking and tracing the sql - agent
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

# this function is implemented in the Flaksapp and only returns the final answer
def get_last_message_content(stream):
    last_message = None
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            last_message = message
        else:
            last_message = message
    
    # Return only the content of the last message
    return last_message.content if last_message else None

In [6]:
# Define the system message for the agent
SQL_PREFIX = """You are an agent designed to interact with a SQL database.
    Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
    Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results if not specified otherwise.
    
    You have access to the following tables: {table_names}
    
    DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.
    
    To start you should ALWAYS look at the tables in the database to see what you can query.
    Then you should query the schema of the most relevant tables.
    """.format(table_names=['notifications','transparency_register','contributions'])

In [7]:
# connect to the database
db_path = os.path.join("..", "db", "notifications.db")
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)

In [8]:
# Set up the language model
memory = MemorySaver()
# Initialize toolkit and agent
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()
system_message = SystemMessage(content=SQL_PREFIX)
graph = create_react_agent(llm, tools, checkpointer=memory, messages_modifier=system_message)

config = {"configurable": {"thread_id": "1"}}

/var/folders/qk/10jspwn51hb5gk7bfj_xq96h0000gn/T/ipykernel_38413/3643827448.py:7: LangGraphDeprecationWarning: Parameter 'messages_modifier' in function 'create_react_agent' is deprecated as of version 0.1.9 and will be removed in version 0.3.0. Use 'state_modifier' parameter instead.
  graph = create_react_agent(llm, tools, checkpointer=memory, messages_modifier=system_message)


## test run
- the following asks three consecutive questions that requires quering three different tables in the sql database

In [10]:
input_list = [{"messages": [("user", "which organisation made most contributions?")]},
              {"messages": [("user", "please list the 5 last contributions that the organisation commented on")]},
              {"messages": [("user", "do you have lobby information on the organisation in the trasparency register?")]}
              ]

for input in input_list:
    response = get_last_message_content(graph.stream(input, config=config, stream_mode="values"))
    print(response)

The organization that made the most contributions is **DIGITALEUROPE** with a total of **24 contributions**.
Here are the 5 last contributions that the organization **DIGITALEUROPE** commented on:

1. **Contribution ID:** 2088
   - **TRIS ID:** 2019-0034-E
   - **Language:** EN
   - **Path:** /digitaleurope_EN.pdf
   - **Full Path:** 2019_0034_E/contributions//digitaleurope_EN.pdf
   - **File Type:** pdf
   - **Call for Do:** 0

2. **Contribution ID:** 2047
   - **TRIS ID:** 2020-0410-F
   - **Language:** EN
   - **Path:** /digitaleurope_EN.pdf
   - **Full Path:** 2020_0410_F/contributions//digitaleurope_EN.pdf
   - **File Type:** pdf
   - **Call for Do:** 1

3. **Contribution ID:** 1384
   - **TRIS ID:** 2020-0813-D
   - **Language:** EN
   - **Path:** /digitaleurope_EN.pdf
   - **Full Path:** 2020_0813_D/contributions//digitaleurope_EN.pdf
   - **File Type:** pdf
   - **Call for Do:** 0

4. **Contribution ID:** 1344
   - **TRIS ID:** 2020-0065-D
   - **Language:** EN
   - **Path:** /